<a href="https://colab.research.google.com/github/ishancoderr/3D_modeling/blob/main/rasterImagePixelCount_for_landsli_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import rasterio
import matplotlib.pyplot as plt
from numpy import asarray

# Open raster and plot
raster_Defomation = rasterio.open("/content/Deformation.tif").read(1)
raster_Aspect = rasterio.open("/content/Aspect.tif").read(1)
raster_Slope = rasterio.open("/content/Slope.tif").read(1)

In [47]:
parameter_set=[]
img_Defomation=asarray(raster_Defomation)
img_Aspect=asarray(raster_Aspect )
img_Slope=asarray(raster_Slope)
parameter_set.append(img_Slope)
parameter_set.append(img_Aspect)
parameter_set.append(img_Defomation)


In [48]:
rows = len(parameter_set[0])
cols = len(parameter_set[0][0])
print(rows,cols)

301 404


In [49]:
new_probabilities={}
parameter_dict=[]
sope_dict={}
aspect_dict={}
defomation_dict={}
parameter_dict.append(sope_dict)
parameter_dict.append(aspect_dict)
parameter_dict.append(defomation_dict)
for num in range(len(parameter_dict)):
  print(num)
  rows = len(parameter_set[num])
  cols = len(parameter_set[num][0])
  for i in range(rows):
    for j in range(cols):
      if parameter_set[num][i,j] in parameter_dict[num]:
        parameter_dict[num][parameter_set[num][i,j]]+=1
      else:
        parameter_dict[num][parameter_set[num][i,j]]= 1

  sum=0
  for i,j in parameter_dict[num].items():
    if(i!=127 and i!=2147483647):
      sum=sum+j

  for i,j in parameter_dict[num].items():
    if(i!=127 and i!=2147483647):
      print(f"{i} value parameter_dict {num}:{j}/{sum}")
      new_probabilities[f'{num}_{i}']=j/sum
print(new_probabilities)

0
2 value parameter_dict 0:17437/18509
4 value parameter_dict 0:627/18509
1 value parameter_dict 0:445/18509
1
3 value parameter_dict 1:4210/18509
1 value parameter_dict 1:9646/18509
2 value parameter_dict 1:4653/18509
2
2 value parameter_dict 2:18495/18509
1 value parameter_dict 2:14/18509
{'0_2': 0.942082230266357, '0_4': 0.03387541196174834, '0_1': 0.024042357771894753, '1_3': 0.22745691285320654, '1_1': 0.52115187206224, '1_2': 0.2513912150845535, '2_2': 0.9992436112161651, '2_1': 0.0007563887838348912}


In [50]:
# Open raster and plot
raster_Defomation_WRA = rasterio.open("/content/Deformation_WRA.tif").read(1)
raster_Aspect_WRA = rasterio.open("/content/Aspect_WRA.tif").read(1)
raster_Slope_WRA = rasterio.open("/content/Slope_WRA.tif").read(1)

In [51]:
parameter_set=[]
img_Defomation_WRA=asarray(raster_Defomation_WRA)
img_Aspect_WRA=asarray(raster_Aspect_WRA)
img_Slope_WRA=asarray(raster_Slope_WRA)
parameter_set.append(raster_Slope_WRA)
parameter_set.append(raster_Aspect_WRA)
parameter_set.append(img_Defomation_WRA)

In [52]:
d={}

for i in range(rows):
  for j in range(cols):
    if img_Defomation_WRA[i,j] in d:
      d[img_Defomation_WRA[i,j]]+=1
    else:
      d[img_Defomation_WRA[i,j]]= 1
print(d)

{2: 104793, 127: 16642, 1: 169}


In [53]:
rows = len(parameter_set[2])
cols = len(parameter_set[2][0])
print(rows,cols)

349 524


In [54]:
slope = [[0 for j in range(cols)] for i in range(rows)]
aspect= [[0 for j in range(cols)] for i in range(rows)]
defomation=[[0 for j in range(cols)] for i in range(rows)]
for layer_index in range(len(parameter_set)):
    for i in range(rows):
        for j in range(cols):
            x = f"{layer_index}_{parameter_set[layer_index][i][j]}"
            if x in new_probabilities.keys():
              if layer_index==0:
                slope[i][j] = new_probabilities[x]
              elif layer_index==1:
                aspect[i][j] = new_probabilities[x]
              elif layer_index==2:
                defomation[i][j] = new_probabilities[x]

final_array=[[0 for j in range(cols)] for i in range(rows)]
for i in range(rows):
  for j in range(cols):
    final_array[i][j]=slope[i][j]*aspect[i][j]*defomation[i][j]



In [55]:
min_value = np.min(final_array)
max_value = np.max(final_array)

# Categorize the values based on the min and max values
for i in range(rows):
    for j in range(cols):
        if final_array[i][j] < min_value + (max_value - min_value) / 4:
            categories[i][j] = 1
        elif final_array[i][j] < min_value + (max_value - min_value) / 2:
            categories[i][j] = 2
        elif final_array[i][j] < min_value + (max_value - min_value) * 3 / 4:
            categories[i][j] = 3
        else:
            categories[i][j] = 4

# Create a colormap for the categories
colormap = {
    1: (255, 0, 0),   # red
    2: (255, 255, 0), # yellow
    3: (255, 255, 255),   # black
    4: (0, 0, 255)    # blue
}

# Create an image from the categories
image_data = np.zeros((rows, cols, 3), dtype=np.uint8)
for i in range(rows):
    for j in range(cols):
        image_data[i][j] = colormap[categories[i][j]]
image = Image.fromarray(image_data)

image.save("categories.png")

